<iframe width="560" height="315" src="https://www.youtube.com/embed/QKMGFCHZH7Y" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [1]:
!pip install sentence-transformers
!pip install protobuf
!pip install deep_translator
!pip install pandas

     |████████████████████████████████| 85 kB 339 kB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 583 kB/s eta 0:00:01
     |████████████████████████████████| 101 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 6.6 MB 48.4 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125918 sha256=c0b1e96fdc9be9636c1aa497677e7908b05e11846023ce6e709ccfd96058392e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: packaging
    Found existing installation: packaging 20.8
    Uninstalling packaging-20.8:
      Successfully uninstalled packaging-20.8
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  

In [2]:
import time

import pandas
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from torch import nn, optim, tensor
from torch.nn import functional as F

In [3]:
class Classifier(nn.Module):
    def __init__(self, embedding_dim, num_labels, dropout):
        super(Classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_labels = num_labels
        self.dropout = dropout

        self.dp = nn.Dropout(self.dropout)
        self.ff = nn.Linear(self.embedding_dim, self.num_labels)

    def forward(self, input_embeddings):
        tensor = self.dp(input_embeddings)
        tensor = self.ff(tensor)
        return tensor, F.softmax(tensor, dim=-1)


class Batcher(object):
    def __init__(self, data_x, data_y, batch_size):
        self.data_x = data_x
        self.data_y = data_y
        self.batch_size = batch_size
        self.n_samples = data_x.shape[0]
        self.indices = torch.randperm(self.n_samples)
        self.ptr = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.ptr > self.n_samples:
            self.ptr = 0
            self.indices = torch.randperm(self.n_samples)
            raise StopIteration
        else:
            batch_indices = self.indices[self.ptr:self.ptr+self.batch_size]
            self.ptr += self.batch_size
            return self.data_x[batch_indices], self.data_y[batch_indices]


data_df = pandas.read_csv("../input/multilingual-spam-data/data-en-hi-de-fr.csv")
     
data_df.dropna(inplace=True)
data_df.drop_duplicates(inplace=True)
data_df.rename(columns={
    "Category": "labels",
    "Message": "text"
}, inplace=True)

le = LabelEncoder()
le.fit(data_df.labels)
data_df["labels"] = le.transform(data_df.labels)

train_x, test_x, train_y, test_y = \
    train_test_split(data_df.text, data_df.labels, stratify=data_df.labels, test_size=0.15,
                     random_state=123)

train_x_de, test_x_de, train_y_de, test_y_de = \
    train_test_split(data_df.text_hi, data_df.labels, stratify=data_df.labels, test_size=0.15,
                     random_state=123)

sentences = train_x.tolist()
test_sentences = test_x_de.tolist()

labels = torch.tensor(train_y.tolist())
test_labels = torch.tensor(test_y_de.tolist())

# encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoder = SentenceTransformer('quora-distilbert-multilingual')
print('Encoding segments...')
start = time.time()
embedding = encoder.encode(sentences, convert_to_tensor=True)
test_sentences_embedding = encoder.encode(test_sentences, convert_to_tensor=True)
print(f"Encoding completed in {time.time() - start} seconds.")

train_batcher = Batcher(embedding, labels, batch_size=16)

num_samples, embeddings_dim = embedding.size()
n_labels = labels.unique().shape[0]

classifier = Classifier(embeddings_dim, n_labels, dropout=0.01)

optimizer = optim.Adam(classifier.parameters())
loss_fn = nn.CrossEntropyLoss()

for e in range(10):
    total_loss = 0
    for batch in train_batcher:
        x, y = batch
        optimizer.zero_grad()
        model_output, prob = classifier(x)
        loss = loss_fn(model_output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'epoch:{e}, total_loss:{total_loss}')

with torch.no_grad():
    model_output, prob = classifier(test_sentences_embedding)
    predictions = torch.argmax(prob, dim=-1)
    results = classification_report(predictions, test_labels)
    print(results)


Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Encoding segments...


Batches:   0%|          | 0/137 [00:00<?, ?it/s]

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

Encoding completed in 161.8462839126587 seconds.
epoch:0, total_loss:62.56313539668918
epoch:1, total_loss:34.271434688940644
epoch:2, total_loss:27.416143734008074
epoch:3, total_loss:23.724894346669316
epoch:4, total_loss:20.943754298612475
epoch:5, total_loss:19.61637761676684
epoch:6, total_loss:17.498989212792367
epoch:7, total_loss:16.614907732000574
epoch:8, total_loss:15.870797763112932
epoch:9, total_loss:14.891708600101992
              precision    recall  f1-score   support

           0       1.00      0.93      0.96       727
           1       0.46      0.94      0.62        47

    accuracy                           0.93       774
   macro avg       0.73      0.93      0.79       774
weighted avg       0.96      0.93      0.94       774

